In [1]:
import numpy as np
import SimpleITK as sitk
import skimage
import os, csv
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


In [2]:
control_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_control_flipped/'
control_imgs = sorted(os.path.join(control_path, fname) for fname in os.listdir(control_path))

control_mask_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_control_seg_flipped/'
control_mask = sorted(os.path.join(control_mask_path, fname) for fname in os.listdir(control_mask_path))

FileNotFoundError: [Errno 2] No such file or directory: '/data/scratch/aalexopoulos/OAI_IW-TSE_control_flipped/'

In [3]:
# FUNCTION THAT APPLIES CANNY EDGE FILTER IN EACH SLICE OF THE INPUT MASK 
# AND RETURNS THE EDGE MATRICES OF 0 AND 1 PIXEL VALUES

def edges_calc(patient_mask_np):
    edges_all = []
    edges_fb = []
    edges_tb = []
    for i in range(patient_mask_np.shape[0]):
        # The threshold is set to 500 bone pixels in order to minimize the misclassified bone pixels 
        # present in the mask that will be used for canny edge detection filters 
        # and thus in the search for min and max coords of tibia and femur bones
        if len(np.where(patient_mask_np[i,:,:]>0.0)[0])>500:
            edges1 = skimage.feature.canny(patient_mask_np[i,:,:],sigma=8,low_threshold=0.05,high_threshold=0.4)
            edges_all.append(edges1)

            patient_mask_np_fb = patient_mask_np[i,:,:].copy() 
            patient_mask_np_fb[np.where(patient_mask_np_fb!=1.0)]=0.0
            edges_2 = skimage.feature.canny(patient_mask_np_fb,sigma=8,low_threshold=0.37,high_threshold=0.39)
            edges_fb.append(edges_2)

            patient_mask_np_tb = patient_mask_np[i,:,:].copy() 
            patient_mask_np_tb[np.where(patient_mask_np_tb!=3.0)]=0.0
            edges_3 = skimage.feature.canny(patient_mask_np_tb,sigma=8,low_threshold=0.9,high_threshold=0.95)
            edges_tb.append(edges_3)
    
    edges_all_np = np.array(edges_all).astype(int)
    edges_fb_np = np.array(edges_fb).astype(int)
    edges_tb_np = np.array(edges_tb).astype(int)

    return edges_all_np, edges_fb_np, edges_tb_np



In [4]:
# FUNCTION THAT CALCULATES THE MINIMUM AND MAXIMUM X AND Y COORDINATES
# FOR THE FEMURAL AND TIBIAL BONES FOR EACH PATIENT

# receives as input femur_edges(,384,384) and tibia_edges(,384,384)
# and returns as output the X,Y coords of Xmin, Xmax for both Femor and Tibia Bones
# and the X,Y coords of Ymin of the Tibial Bone

# Here the thresholds are held in ZERO for the number of ones pixel values for the Canny Edge Images
# because the minimization of misclassified bone pixels was applied in the Canny Edge calculation function

def max_min_coords(femur_edges, tibia_edges):
    #firstly find min and max X coords of FEMUR for each slice mask
    ind_fb_xmin = []
    ind_fb_xmax = []
    for i in range(femur_edges.shape[0]):
        if len(np.where(femur_edges[i,:,:]==1.0)[0])>0:
            indices_1 = np.where(femur_edges[i,:,:]==1.0)
            indices_1 = np.array(indices_1)
            sorted_indices_1 = sorted((tp1 for tp1 in zip(*indices_1)), key=lambda x:x[1])
            ind_fb_xmin.append(sorted_indices_1[0])
            ind_fb_xmax.append(sorted_indices_1[-1])
    
    #secondly find min X coords of FEMUR for one input patient mask
    ind_fb_xmin_np = np.array(ind_fb_xmin)
    if ind_fb_xmin_np.ndim>1:    
        ind_fb_xmin_global = ind_fb_xmin_np[np.where(ind_fb_xmin_np[:,1]==np.amin(ind_fb_xmin_np[:,1])),:]
        if ind_fb_xmin_global.shape[1] == 1:
            fb_xmin_global_xcoord = ind_fb_xmin_global[0][0][1]
            fb_xmin_global_ycoord = ind_fb_xmin_global[0][0][0]
        if ind_fb_xmin_global.shape[1] > 1:
            idx = np.where(ind_fb_xmin_global[:,:,0] == np.min(ind_fb_xmin_global[:,:,0]))
            fb_xmin_global_xcoord = ind_fb_xmin_global[idx][0][1]
            fb_xmin_global_ycoord = ind_fb_xmin_global[idx][0][0]
    xmin_femur_coords = np.array([fb_xmin_global_xcoord,fb_xmin_global_ycoord])

    #thirdly find max X coords of FEMUR for one input patient mask
    ind_fb_xmax_np = np.array(ind_fb_xmax)
    if ind_fb_xmax_np.ndim>1:
        ind_fb_xmax_global = ind_fb_xmax_np[np.where(ind_fb_xmax_np[:,1]==np.amax(ind_fb_xmax_np[:,1])),:]
        if ind_fb_xmax_global.shape[1] == 1:
            fb_xmax_global_xcoord = ind_fb_xmax_global[0][0][1]
            fb_xmax_global_ycoord = ind_fb_xmax_global[0][0][0]
        if ind_fb_xmax_global.shape[1] > 1:
            idx = np.where(ind_fb_xmax_global[:,:,0] == np.max(ind_fb_xmax_global[:,:,0]))
            fb_xmax_global_xcoord = ind_fb_xmax_global[idx][0][1]
            fb_xmax_global_ycoord = ind_fb_xmax_global[idx][0][0]
    xmax_femur_coords = np.array([fb_xmax_global_xcoord, fb_xmax_global_ycoord])

    # NOW THE SAME FOR TIBIAL BONE
    # Firstly find all min and max values for X coord of TIBIA for each slice
    ind_tb_xmin = []
    ind_tb_xmax = []
    ind_tb_ymin = []
    for i in range(tibia_edges.shape[0]):
        if len(np.where(tibia_edges[i,:,:]==1.0)[0])>0:
            indices_1 = np.where(tibia_edges[i,:,:]==1.0)
            indices_1 = np.array(indices_1)
            sorted_indices_1 = sorted((tp1 for tp1 in zip(*indices_1)), key=lambda x:x[1])
            ind_tb_xmin.append(sorted_indices_1[0])
            ind_tb_xmax.append(sorted_indices_1[-1])
            sorted_indices_2 = sorted((tp2 for tp2 in zip(*indices_1)), key=lambda x:x[0])
            ind_tb_ymin.append(sorted_indices_2[0])
    
    # Secondly find min values for X coord of TIBIA for one input patient mask
    ind_tb_xmin_np = np.array(ind_tb_xmin)
    if ind_tb_xmin_np.ndim>1:
        ind_tb_xmin_global = ind_tb_xmin_np[np.where(ind_tb_xmin_np[:,1]==np.amin(ind_tb_xmin_np[:,1])),:]
        if ind_tb_xmin_global.shape[1]==1:    
            tb_xmin_global_xcoord = ind_tb_xmin_global[0][0][1]
            tb_xmin_global_ycoord = ind_tb_xmin_global[0][0][0]
        if ind_tb_xmin_global.shape[1]>1:
            idx = np.where(ind_tb_xmin_global[:,:,0]==np.min(ind_tb_xmin_global[:,:,0]))
            tb_xmin_global_xcoord = ind_tb_xmin_global[idx][0][1]
            tb_xmin_global_ycoord = ind_tb_xmin_global[idx][0][0]
    xmin_tibia_coords = np.array([tb_xmin_global_xcoord,tb_xmin_global_ycoord])

    # Thirdly find max values for X coord of TIBIA for one input patient mask
    ind_tb_xmax_np = np.array(ind_tb_xmax)
    if ind_tb_xmax_np.ndim>1:
        ind_tb_xmax_global = ind_tb_xmax_np[np.where(ind_tb_xmax_np[:,1]==np.amax(ind_tb_xmax_np[:,1])),:]
        if ind_tb_xmax_global.shape[1]==1 :
            tb_xmax_global_xcoord = ind_tb_xmax_global[0][0][1]
            tb_xmax_global_ycoord = ind_tb_xmax_global[0][0][0]
        if ind_tb_xmax_global.shape[1]>1 :
            idx = np.where(ind_tb_xmax_global[:,:,0]==np.max(ind_tb_xmax_global[:,:,0]))
            tb_xmax_global_xcoord = ind_tb_xmax_global[idx][0][1]
            tb_xmax_global_ycoord = ind_tb_xmax_global[idx][0][0]
    xmax_tibia_coords = np.array([tb_xmax_global_xcoord,tb_xmax_global_ycoord])

    # Fourthly find min values for Y coord of TIBIA for one input patient mask
    ind_tb_ymin_np = np.array(ind_tb_ymin)
    if ind_tb_ymin_np.ndim>1:
        ind_tb_ymin_global = ind_tb_ymin_np[np.where(ind_tb_ymin_np[:,0]==np.amin(ind_tb_ymin_np[:,0])),:]
        if ind_tb_ymin_global.shape[1]==1:    
            tb_ymin_global_xcoord = ind_tb_ymin_global[0][0][1]
            tb_ymin_global_ycoord = ind_tb_ymin_global[0][0][0]
        if ind_tb_ymin_global.shape[1]>1:
            idx = np.where(ind_tb_ymin_global[:,:,1]==np.amin(ind_tb_ymin_global[:,:,1]))
            tb_ymin_global_xcoord = ind_tb_ymin_global[idx][0][1]
            tb_ymin_global_ycoord = ind_tb_ymin_global[idx][0][0]
    ymin_tibia_coords = np.array([tb_ymin_global_xcoord,tb_ymin_global_ycoord])

    return xmin_femur_coords, xmax_femur_coords, xmin_tibia_coords, xmax_tibia_coords, ymin_tibia_coords

In [5]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
control_femural_edges_50 = []
control_tibial_edges_50 = []

control_fb_xmin_c_50 = []
control_fb_xmax_c_50 = []
control_tb_xmin_c_50 = []
control_tb_xmax_c_50 = []
control_tb_ymin_c_50 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

control_all_coords_50 = np.empty((0,11),dtype=int)


In [6]:
k = 0

for i in range(0,len(control_imgs[:50])):
    p_num = control_imgs[i].split('/')[-1]
    mask = sitk.ReadImage(control_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(control_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    control_femural_edges_50.append(fb_edges)
    control_tibial_edges_50.append(tb_edges)

    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)
    
    control_fb_xmin_c_50.append(fb_xmin_c)
    control_fb_xmax_c_50.append(fb_xmax_c)
    control_tb_xmin_c_50.append(tb_xmin_c)
    control_tb_xmax_c_50.append(tb_xmax_c)
    control_tb_ymin_c_50.append(tb_ymin_c)
    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    control_all_coords_50 = np.append(control_all_coords_50,all_info,axis=0)
    
    print(control_imgs[i].split('/')[-1],k)
    k = k +1


NameError: name 'control_imgs' is not defined

In [ ]:
control_fb_xmax_c_all_patients_np = np.array(control_fb_xmax_c_all_patients)
control_fb_xmin_c_all_patients_np = np.array(control_fb_xmin_c_all_patients)
control_tb_xmax_c_all_patients_np = np.array(control_tb_xmax_c_all_patients)
control_tb_xmin_c_all_patients_np = np.array(control_tb_xmin_c_all_patients)
control_tb_ymin_c_all_patients_np = np.array(control_tb_ymin_c_all_patients)

In [ ]:
save_path_50 = '/data/scratch/aalexopoulos/coords_outputs/all_control_50_coords.csv'
np.savetxt(save_path_50,control_all_coords_50,delimiter=',')


In [94]:
fb_xmax_c_global = control_fb_xmax_c_50_np[np.where(control_fb_xmax_c_50_np[:,0] == np.amax(control_fb_xmax_c_50_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global control femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
print(control_imgs[int(np.where(control_fb_xmax_c_50_np[:,0] == np.amax(control_fb_xmax_c_50_np[:,0]))[0])].split('/')[-1])

fb_xmin_c_global = control_fb_xmin_c_50_np[np.where(control_fb_xmin_c_50_np[:,0] == np.amin(control_fb_xmin_c_50_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global control femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
print(control_imgs[int(np.where(control_fb_xmin_c_50_np[:,0] == np.amin(control_fb_xmin_c_50_np[:,0]))[0])].split('/')[-1])

tb_xmax_c_global = control_tb_xmax_c_50_np[np.where(control_tb_xmax_c_50_np[:,0] == np.amax(control_tb_xmax_c_50_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global control tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_xmax_c_50_np[:,0] == np.amax(control_tb_xmax_c_50_np[:,0]))[0])].split('/')[-1])

tb_xmin_c_global = control_tb_xmin_c_50_np[np.where(control_tb_xmin_c_50_np[:,0] == np.amin(control_tb_xmin_c_50_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global control tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_xmin_c_50_np[:,0] == np.amin(control_tb_xmin_c_50_np[:,0]))[0])].split('/')[-1])

tb_ymin_c_global = control_tb_ymin_c_50_np[np.where(control_tb_ymin_c_50_np[:,1] == np.amin(control_tb_ymin_c_50_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global control tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_ymin_c_50_np[:,1] == np.amin(control_tb_ymin_c_50_np[:,1]))[0])].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

Global control femural xmax x coord: 350, y coord: 157
Global control femural xmin x coord: 59, y coord: 128
Global control tibial xmax x coord: 340, y coord: 269
Global control tibial xmin x coord: 66, y coord: 235
Global control tibial ymin x coord: 153, y coord: 212


In [ ]:
global_control_coords = {'Global Control FB xmax x coord':fb_xmax_c_global_xcoord,'Global Control FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global Control FB xmin x coord':fb_xmin_c_global_xcoord,'Global Control FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global Control TB xmax x coord':tb_xmax_c_global_xcoord,'Global Control tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global Control tb xmin x coord':tb_xmin_c_global_xcoord,'Global Control tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global Control tb ymin x coord':tb_ymin_c_global_xcoord,'Global Control tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/data/scratch/aalexopoulos/coords_outputs/global_control_50_coords.csv','w') as f:
    for key in global_control_coords.keys():
        f.write("%s, %s\n" % (key, global_control_coords[key]))

In [95]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR CONTROL GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

[[ 66 235]
 [ 59 128]]
Rect xmin coord: 59, ymin coord: 128
[[340 269]
 [350 157]]
Rect xmax coord: 350, ymax coord: 269
Rect Width: 291, Rect Height: 141


In [ ]:
rect_control_coords = {'control rect xmin coord': rect_xmin_global,'control rect ymin coord': rect_ymin_global, 
                        'control rect xmax coord': rect_xmax_global,'control rect ymax coord': rect_ymax_global,
                        'control rect width': rect_width,'control rect height': rect_height }
with open('/data/scratch/aalexopoulos/coords_outputs/rect_control_100_coords.csv','w') as f:
    for key in rect_control_coords.keys():
        f.write("%s, %s\n" % (key, rect_control_coords[key]))

### HERE 50-100 ###

In [ ]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
control_femural_edges_50_100 = []
control_tibial_edges_50_100 = []

control_fb_xmin_c_50_100 = []
control_fb_xmax_c_50_100 = []
control_tb_xmin_c_50_100 = []
control_tb_xmax_c_50_100 = []
control_tb_ymin_c_50_100 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

control_all_coords_50_100 = np.empty((0,11),dtype=int)


In [ ]:
k = 0

for i in range(50,len(control_imgs[:100])):
    p_num = control_imgs[i].split('/')[-1]
    mask = sitk.ReadImage(control_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(control_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    control_femural_edges_50_100.append(fb_edges)
    control_tibial_edges_50_100.append(tb_edges)

    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)
    
    control_fb_xmin_c_50_100.append(fb_xmin_c)
    control_fb_xmax_c_50_100.append(fb_xmax_c)
    control_tb_xmin_c_50_100.append(tb_xmin_c)
    control_tb_xmax_c_50_100.append(tb_xmax_c)
    control_tb_ymin_c_50_100.append(tb_ymin_c)
    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    control_all_coords_50_100 = np.append(control_all_coords_50_100,all_info,axis=0)
    
    print(control_imgs[i].split('/')[-1],k)
    k = k +1


NameError: name 'control_imgs' is not defined

In [ ]:
control_fb_xmax_c_all_patients_np = np.array(control_fb_xmax_c_all_patients)
control_fb_xmin_c_all_patients_np = np.array(control_fb_xmin_c_all_patients)
control_tb_xmax_c_all_patients_np = np.array(control_tb_xmax_c_all_patients)
control_tb_xmin_c_all_patients_np = np.array(control_tb_xmin_c_all_patients)
control_tb_ymin_c_all_patients_np = np.array(control_tb_ymin_c_all_patients)

In [ ]:
save_path_50_100 = '/data/scratch/aalexopoulos/coords_outputs/all_control_50_100_coords.csv'
np.savetxt(save_path_50_100,control_all_coords_50_100,delimiter=',')


In [ ]:
fb_xmax_c_global = control_fb_xmax_c_50_100_np[np.where(control_fb_xmax_c_50_100_np[:,0] == np.amax(control_fb_xmax_c_50_100_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global control femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
print(control_imgs[int(np.where(control_fb_xmax_c_50_100_np[:,0] == np.amax(control_fb_xmax_c_50_100_np[:,0]))[0])].split('/')[-1])

fb_xmin_c_global = control_fb_xmin_c_50_100_np[np.where(control_fb_xmin_c_50_100_np[:,0] == np.amin(control_fb_xmin_c_50_100_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global control femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
print(control_imgs[int(np.where(control_fb_xmin_c_50_100_np[:,0] == np.amin(control_fb_xmin_c_50_100_np[:,0]))[0])].split('/')[-1])

tb_xmax_c_global = control_tb_xmax_c_50_100_np[np.where(control_tb_xmax_c_50_100_np[:,0] == np.amax(control_tb_xmax_c_50_100_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global control tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_xmax_c_50_100_np[:,0] == np.amax(control_tb_xmax_c_50_100_np[:,0]))[0])].split('/')[-1])

tb_xmin_c_global = control_tb_xmin_c_50_100_np[np.where(control_tb_xmin_c_50_100_np[:,0] == np.amin(control_tb_xmin_c_50_100_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global control tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_xmin_c_50_100_np[:,0] == np.amin(control_tb_xmin_c_50_100_np[:,0]))[0])].split('/')[-1])

tb_ymin_c_global = control_tb_ymin_c_50_100_np[np.where(control_tb_ymin_c_50_100_np[:,1] == np.amin(control_tb_ymin_c_50_100_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global control tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
print(control_imgs[int(np.where(control_tb_ymin_c_50_100_np[:,1] == np.amin(control_tb_ymin_c_50_100_np[:,1]))[0])].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

Global control femural xmax x coord: 350, y coord: 157
Global control femural xmin x coord: 59, y coord: 128
Global control tibial xmax x coord: 340, y coord: 269
Global control tibial xmin x coord: 66, y coord: 235
Global control tibial ymin x coord: 153, y coord: 212


In [ ]:
global_control_coords = {'Global Control FB xmax x coord':fb_xmax_c_global_xcoord,'Global Control FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global Control FB xmin x coord':fb_xmin_c_global_xcoord,'Global Control FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global Control TB xmax x coord':tb_xmax_c_global_xcoord,'Global Control tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global Control tb xmin x coord':tb_xmin_c_global_xcoord,'Global Control tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global Control tb ymin x coord':tb_ymin_c_global_xcoord,'Global Control tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/data/scratch/aalexopoulos/coords_outputs/global_control_100_coords.csv','w') as f:
    for key in global_control_coords.keys():
        f.write("%s, %s\n" % (key, global_control_coords[key]))

In [ ]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR CONTROL GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

[[ 66 235]
 [ 59 128]]
Rect xmin coord: 59, ymin coord: 128
[[340 269]
 [350 157]]
Rect xmax coord: 350, ymax coord: 269
Rect Width: 291, Rect Height: 141


In [ ]:
rect_control_coords = {'control rect xmin coord': rect_xmin_global,'control rect ymin coord': rect_ymin_global, 
                        'control rect xmax coord': rect_xmax_global,'control rect ymax coord': rect_ymax_global,
                        'control rect width': rect_width,'control rect height': rect_height }
with open('/data/scratch/aalexopoulos/coords_outputs/rect_control_100_coords.csv','w') as f:
    for key in rect_control_coords.keys():
        f.write("%s, %s\n" % (key, rect_control_coords[key]))